In [1]:
from blob_storage import download as blob_storage 
import pandas as pd
import numpy as np
import util
import sentiment_score

In [2]:
local_path = "C:\\Users\\Vinim\\Documents\\github\\tcc_vinicius\\data"

# download de todos os arquivos 
blob_storage.download_blob_storage(local_path)


Listing blobs...


0it [00:00, ?it/s]

26it [00:09,  2.67it/s]

26 arquivos baixados com sucesso


In [3]:
df = util.get_dataframe(local_path)

In [ ]:
data\

In [ ]:
pd.read_parquet('C:\\Users\\Vinim\\Documents\\github\\tcc_vinicius\\data\\result_2024-02-05-21h-42min.parquet')

In [37]:
df = df[df['data_de_busca'] == '2024-02-05'].copy()

- sentiment score

In [38]:
liwc2015 = sentiment_score.read_liwc('LIWC2015 Dictionary - Brazilian Portuguese.dicx')

In [39]:
# df['full_text_score'] = df['full_text'].apply(lambda x: sentiment_score.get_sentiment_score(x, liwc2015))

In [43]:
import nltk

In [44]:
def get_sentiment_score(text, dictionary):
    tokens = nltk.word_tokenize(text,language='portuguese')

    df_tokens = pd.DataFrame({'tokens':tokens})
    df_tokens['tokens'] = df_tokens['tokens'].astype('str').str.lower()

    result = pd.merge(df_tokens, dictionary, left_on='tokens',right_on='DicTerm', how='left')
    result['score_liwc2015'].fillna(0, inplace=True)

    palavras = result.shape[0]
    count_n = (result['score_liwc2015'] == -1).sum()
    count_p = (result['score_liwc2015'] == 1).sum()

    result_score = result.loc[result['score_liwc2015'] != 0,'score_liwc2015'].mean()

    raw_mean = result['score_liwc2015'].mean()

    return f'{count_p}-{count_n}-{result_score}-{palavras}-{raw_mean}'

In [45]:
df['scores'] = df['full_text'].apply(lambda x: get_sentiment_score(x, liwc2015))

In [48]:
df[['positivo', 'negativo', 'média', 'qnt palavras', 'média todos']] = df['scores'].str.split('-', expand=True)

In [49]:
df.drop_duplicates(subset='name').to_excel('teste_score_05-02.xlsx')